In [1]:
!pip install "protobuf==3.20.*" --force-reinstall
!pip install -q "monai[nibabel]" --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-m

In [2]:
import os, random, numpy as np, nibabel as nib, time
from tqdm.auto import tqdm
from glob import glob

import torch, torch.nn as nn
from torch.utils.data import DataLoader

import monai
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Orientationd, Spacingd,
    CropForegroundd, NormalizeIntensityd, ResizeWithPadOrCropd,
    ConcatItemsd, EnsureTyped
)
from monai.data import Dataset, CacheDataset
from monai.networks.nets import SwinUNETR
from monai.losses import DiceCELoss, FocalLoss, TverskyLoss
from monai.inferers import sliding_window_inference

print("MONAI:", monai.__version__)
print("Torch:", torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


<frozen importlib._bootstrap_external>:1241: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
2025-12-07 16:13:15.332258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765123995.501938      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765123995.551793      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


MONAI: 1.5.1
Torch: 2.6.0+cu124
Device: cuda


In [3]:
BASE = "/kaggle/input/brats20-dataset-training-validation"

TRAIN_DIR = f"{BASE}/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData"
VAL_DIR   = f"{BASE}/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData"
TEST_DIR  = VAL_DIR  # No labels provided, will use for inference only

IMG_SIZE = (96, 96, 96)
IN_CHANNELS = 4
OUT_CHANNELS = 4   # 0,1,2,3 (we remap 4->3)
BATCH_SIZE = 2
EPOCHS = 10
LR = 1e-4

OUTDIR = "/kaggle/working/swin_unetr_run"
CKPT_DIR = f"{OUTDIR}/checkpoints"; os.makedirs(CKPT_DIR, exist_ok=True)
PRED_DIR = f"{OUTDIR}/predictions"; os.makedirs(PRED_DIR, exist_ok=True)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

print("OUTPUT DIR:", OUTDIR)


OUTPUT DIR: /kaggle/working/swin_unetr_run


In [4]:
def load_brats_train(root):
    items = []
    for case in sorted(os.listdir(root)):
        folder = f"{root}/{case}"

        # Skip junk files / CSV / non-directories
        if not os.path.isdir(folder): 
            continue
        if case.endswith(".csv") or case.startswith("."):
            continue
        
        cid = case
        
        f = f"{folder}/{cid}_flair.nii"
        t1 = f"{folder}/{cid}_t1.nii"
        t1ce = f"{folder}/{cid}_t1ce.nii"
        t2 = f"{folder}/{cid}_t2.nii"
        seg = f"{folder}/{cid}_seg.nii"

        # Make sure ALL 4 modalities + seg exist
        if os.path.exists(f) and os.path.exists(t1) and os.path.exists(t1ce) and os.path.exists(t2) and os.path.exists(seg):
            items.append({"id": cid, "flair": f, "t1": t1, "t1ce": t1ce, "t2": t2, "seg": seg})
    
    return items


def load_brats_test(root):
    items = []
    for case in sorted(os.listdir(root)):
        folder = f"{root}/{case}"
        if not os.path.isdir(folder): 
            continue
        if case.endswith(".csv") or case.startswith("."):
            continue

        cid = case
        f = f"{folder}/{cid}_flair.nii"
        t1 = f"{folder}/{cid}_t1.nii"
        t1ce = f"{folder}/{cid}_t1ce.nii"
        t2 = f"{folder}/{cid}_t2.nii"

        if os.path.exists(f) and os.path.exists(t1) and os.path.exists(t1ce) and os.path.exists(t2):
            items.append({"id": cid, "flair": f, "t1": t1, "t1ce": t1ce, "t2": t2})
    return items


train_files = load_brats_train(TRAIN_DIR)
test_files  = load_brats_test(TEST_DIR)

print("Train cases:", len(train_files))
print("Test cases:", len(test_files))


Train cases: 368
Test cases: 125


In [5]:
train_transforms = monai.transforms.Compose([
    LoadImaged(keys=["flair","t1","t1ce","t2","seg"]),
    EnsureChannelFirstd(keys=["flair","t1","t1ce","t2","seg"]),
    Orientationd(keys=["flair","t1","t1ce","t2","seg"], axcodes="RAS"),
    Spacingd(keys=["flair","t1","t1ce","t2","seg"],
             pixdim=(1,1,1), mode=("bilinear","bilinear","bilinear","bilinear","nearest")),
    CropForegroundd(keys=["flair","t1","t1ce","t2","seg"], source_key="flair"),
    ResizeWithPadOrCropd(keys=["flair","t1","t1ce","t2","seg"], spatial_size=IMG_SIZE),
    NormalizeIntensityd(keys=["flair","t1","t1ce","t2"], nonzero=True, channel_wise=True),
    ConcatItemsd(keys=["flair","t1","t1ce","t2"], name="image"),
    EnsureTyped(keys=["image","seg"])
])

test_transforms = monai.transforms.Compose([
    LoadImaged(keys=["flair","t1","t1ce","t2"]),
    EnsureChannelFirstd(keys=["flair","t1","t1ce","t2"]),
    Orientationd(keys=["flair","t1","t1ce","t2"], axcodes="RAS"),
    Spacingd(keys=["flair","t1","t1ce","t2"], pixdim=(1,1,1),
             mode=("bilinear","bilinear","bilinear","bilinear")),
    CropForegroundd(keys=["flair","t1","t1ce","t2"], source_key="flair"),
    ResizeWithPadOrCropd(keys=["flair","t1","t1ce","t2"], spatial_size=IMG_SIZE),
    NormalizeIntensityd(keys=["flair","t1","t1ce","t2"], nonzero=True, channel_wise=True),
    ConcatItemsd(keys=["flair","t1","t1ce","t2"], name="image"),
    EnsureTyped(keys=["image"])
])


/usr/local/lib/python3.11/dist-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.spatial.dictionary Orientationd.__init__:labels: Current default value of argument `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` was changed in version None from `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` to `labels=None`. Default value changed to None meaning that the transform now uses the 'space' of a meta-tensor, if applicable, to determine appropriate axis labels.
  warn_deprecated(argname, msg, warning_category)


In [ ]:
train_ds = CacheDataset(train_files, transform=train_transforms, cache_rate=0.7)
test_ds  = Dataset(test_files, transform=test_transforms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(train_ds, batch_size=1, shuffle=False)  # since training has labels
test_loader  = DataLoader(test_ds, batch_size=1, shuffle=False)

print("Batches => Train:", len(train_loader), "Val:", len(val_loader), "Test:", len(test_loader))


Loading dataset:  19%|█▉        | 50/257 [02:02<08:58,  2.60s/it]

In [ ]:
model = SwinUNETR(
       
    in_channels=4,
    out_channels=4,
    feature_size=48,
    use_checkpoint=True
).to(device)


loss_dicece = DiceCELoss(include_background=False, to_onehot_y=True, softmax=True)
loss_tversky = TverskyLoss(alpha=0.3, beta=0.7, to_onehot_y=True, softmax=True)
loss_focal = FocalLoss(include_background=False, to_onehot_y=True, gamma=2.0)

def hybrid_loss(pred, target):
    target[target == 4] = 3
    l1 = loss_dicece(pred, target)
    l2 = loss_tversky(pred, target)
    l3 = loss_focal(pred, target)
    return 0.4*l1 + 0.3*l2 + 0.3*l3

optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
scaler = torch.cuda.amp.GradScaler()


In [ ]:
def dice_region(pred, gt):
    pred = pred.cpu().numpy()[0]
    gt   = gt.cpu().numpy()[0]
    gt[gt==4] = 3

    eps = 1e-7
    scores = {}
    scores["WT"] = (2*np.sum((pred>0)&(gt>0))) / (np.sum(pred>0)+np.sum(gt>0)+eps)
    scores["TC"] = (2*np.sum(((pred==1)|(pred==3)) & ((gt==1)|(gt==3)))) / (
                   np.sum((pred==1)|(pred==3))+np.sum((gt==1)|(gt==3))+eps)
    scores["ET"] = (2*np.sum((pred==3)&(gt==3))) / (np.sum(pred==3)+np.sum(gt==3)+eps)
    return scores


ckpt = torch.load(f"{CKPT_DIR}/best_swin_hybrid.pth", map_location=device)
model.load_state_dict(ckpt["model_state"])
optimizer.load_state_dict(ckpt["optimizer_state"])
start_epoch = ckpt["epoch"] + 1
print("🔄 Resuming Training from Epoch", start_epoch)
for epoch in range(start_epoch, EPOCHS+1):


In [ ]:
best_dice = 0
per_class_dice_history = []   
train_losses = []             

for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss = 0

    # ================== TRAIN ==================
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
    for batch in pbar:
        img = batch["image"].to(device)
        seg = batch["seg"].to(device)

        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            pred = model(img)
            loss = hybrid_loss(pred, seg)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        pbar.set_postfix({"loss": f"{running_loss/(pbar.n+1):.4f}"})

    # ================== VALIDATION ==================
    model.eval()
    dices = {"WT":[], "TC":[], "ET":[]}

    with torch.no_grad():
        vbar = tqdm(val_loader, desc="Validating", leave=False)
        for v in vbar:
            img = v["image"].to(device)
            seg = v["seg"].to(device)

            out = sliding_window_inference(img, IMG_SIZE, 1, model)
            pred = torch.argmax(out.softmax(1), 1, keepdim=True)
            d = dice_region(pred, seg)
            for k in d:
                dices[k].append(d[k])

    mean_dice = np.mean([np.mean(v) for v in dices.values()])
    print(f"🎯 Dice -> WT:{np.mean(dices['WT']):.4f}, TC:{np.mean(dices['TC']):.4f}, ET:{np.mean(dices['ET']):.4f}, Mean:{mean_dice:.4f}")

    # ---- Save epoch metrics ----
    per_class_dice_history.append([
        np.mean(dices["WT"]),
        np.mean(dices["TC"]),
        np.mean(dices["ET"])
    ])
    train_losses.append(running_loss/(len(train_loader)))

    # ---- Save BEST Model with optimizer ----
    if mean_dice > best_dice:
        best_dice = mean_dice
        torch.save({
            "epoch": epoch,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "best_dice": best_dice
        }, f"{CKPT_DIR}/best_swin_hybrid.pth")
        print("💾 Saved BEST model (with optimizer)!")

# ================== SAVE LAST EPOCH MODEL ==================
torch.save(model.state_dict(), f"{CKPT_DIR}/last_epoch_swin_hybrid.pth")
print("📌 Saved LAST epoch model!")

# ================== SAVE TRAINING METRICS ==================
import pandas as pd

df = pd.DataFrame(per_class_dice_history, columns=["WT","TC","ET"])
df["Mean"] = df.mean(axis=1)
df["Train_Loss"] = train_losses
df.to_csv(f"{OUTDIR}/metrics_log.csv", index_label="Epoch")

print("📊 Metrics saved to:", f"{OUTDIR}/metrics_log.csv")


In [ ]:
import torch

# Define device again (if needed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define checkpoint directory
CKPT_DIR = "/kaggle/working/swin_unetr_run"

# ===== LOAD BEST CHECKPOINT =====
ckpt = torch.load(f"{CKPT_DIR}/best_swin_hybrid.pth", map_location=device)
model.load_state_dict(ckpt["model_state"])
model.to(device)
model.eval()

print("✔ Loaded Trained SwinUNETR Model!")


In [ ]:
import numpy as np
import torch.serialization as serialization

# allow numpy scalar (needed in PyTorch 2.6+)
serialization.add_safe_globals([np.generic])

# Load checkpoint safely
ckpt = torch.load(
    f"{CKPT_DIR}/best_swin_hybrid.pth", 
    map_location=device, 
    weights_only=False
)

# Restore model
model.load_state_dict(ckpt["model_state"])
model.eval()


In [ ]:
for batch in tqdm(test_loader, desc="Inference"):
    pid = batch["id"][0]
    img = batch["image"].to(device)

    out = sliding_window_inference(img, IMG_SIZE, 1, model)
    pred = torch.argmax(out.softmax(1), 1).cpu().numpy()[0]

    affine = np.eye(4)
    for key in ["flair","t1","t1ce","t2"]:
        if key in batch and hasattr(batch[key][0], "meta"):
            try:
                affine = nib.load(batch[key][0].meta["filename_or_obj"]).affine
                break
            except:
                pass

    nib.save(
        nib.Nifti1Image(pred.astype(np.uint8), affine),
        f"{PRED_DIR}/{pid}_pred.nii.gz"
    )


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# ===================== SAVE RESULTS =====================
df = pd.DataFrame(per_class_dice_history, columns=["WT","TC","ET"])
df["Mean"] = df.mean(axis=1)
df["Train_Loss"] = train_losses
df.to_csv(f"{OUTDIR}/metrics_log.csv", index_label="Epoch")

print("\n📌 Metrics saved at:", f"{OUTDIR}/metrics_log.csv")

# ===================== PLOT DICE CURVES =====================
plt.figure(figsize=(10,5))
plt.plot(df["WT"], label="WT")
plt.plot(df["TC"], label="TC")
plt.plot(df["ET"], label="ET")
plt.plot(df["Mean"], label="Mean", linewidth=3, color="black")
plt.title("Dice Score Progress")
plt.xlabel("Epoch")
plt.ylabel("Dice")
plt.legend()
plt.grid()
plt.savefig(f"{OUTDIR}/dice_curve.png", dpi=200)
plt.show()

# ===================== PLOT LOSS =====================
plt.figure(figsize=(6,4))
plt.plot(df["Train_Loss"], marker="o")
plt.title("Training Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.savefig(f"{OUTDIR}/loss_curve.png", dpi=200)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

# --------------------------
# Utility for Dice
# --------------------------
def dice_coef(pred, gt, cls):
    pred_c = (pred == cls)
    gt_c   = (gt == cls)
    if gt_c.sum() == 0 and pred_c.sum() == 0:
        return 1.0
    return (2 * np.logical_and(pred_c, gt_c).sum()) / (pred_c.sum() + gt_c.sum() + 1e-8)

# --------------------------
# PICK SAMPLE
# --------------------------
sample = next(iter(test_loader))
img = sample["image"].to(device)

with torch.no_grad():
    out = sliding_window_inference(img, IMG_SIZE, 1, model)
pred = torch.argmax(out.softmax(1), 1).cpu().numpy()[0]

gt = sample["seg"][0,0].cpu().numpy()
slice_id = np.argmax((gt > 0).sum(axis=(1,2)))  # slice with max tumor

mri_slice = img[0,0,slice_id].cpu().numpy()
pred_slice = pred[slice_id]
gt_slice = gt[slice_id]
error_map = np.abs(gt_slice - pred_slice)

# --------------------------
# DICE SCORES
# --------------------------
dice_ET = dice_coef(pred, gt, 3)
dice_TC = dice_coef(pred, gt, 2)
dice_WT = dice_coef(pred, gt, 1)
dice_BG = dice_coef(pred, gt, 0)

# --------------------------
# PLOT
# --------------------------
plt.figure(figsize=(16,12))

# 1) MRI
plt.subplot(3,4,1)
plt.title("Original MRI Image")
plt.imshow(mri_slice, cmap='gray')
plt.axis('off')

# 2) Ground truth
plt.subplot(3,4,2)
plt.title("Ground Truth Mask")
plt.imshow(gt_slice, cmap='tab20')
plt.axis('off')

# 3) Prediction
plt.subplot(3,4,3)
plt.title("Predicted Mask")
plt.imshow(pred_slice, cmap='tab20')
plt.axis('off')

# 4) Error Map
plt.subplot(3,4,4)
plt.title("Error Map (|GT - Pred|)")
plt.imshow(error_map, cmap='Reds')
plt.colorbar(label="Error Magnitude")
plt.axis('off')

# --------------------------
# Per-Class Overlays
# --------------------------
def masked_overlay(img, mask, cls, title):
    base = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    base = np.uint8(base); base = np.stack([base]*3, axis=-1)
    m = np.uint8(mask == cls)
    m = cv2.resize(m, (base.shape[1], base.shape[0]))
    color = np.zeros_like(base)
    if cls == 3:  color[:] = (255, 0, 0)     # ET = Red
    if cls == 2:  color[:] = (0, 255, 0)     # TC = Green
    if cls == 1:  color[:] = (0, 0, 255)     # WT = Blue
    overlay = cv2.addWeighted(base, 0.6, color, 0.4, 0)
    plt.imshow(np.where(m[...,None]==1, overlay, base))
    plt.title(title)
    plt.axis('off')

# ET
plt.subplot(3,4,5)
masked_overlay(mri_slice, gt_slice, 3, f"ET (Enhancing Tumor)\nDice: {dice_ET:.3f}")

# TC
plt.subplot(3,4,6)
masked_overlay(mri_slice, gt_slice, 2, f"TC (Tumor Core)\nDice: {dice_TC:.3f}")

# WT
plt.subplot(3,4,7)
masked_overlay(mri_slice, gt_slice, 1, f"WT (Whole Tumor)\nDice: {dice_WT:.3f}")

# Background
plt.subplot(3,4,8)
masked_overlay(mri_slice, gt_slice, 0, f"Background\nDice: {dice_BG:.3f}")

# --------------------------
# TEXT BOX SUMMARY
# --------------------------
plt.subplot(3,4,9)
plt.axis('off')
text = f"""
📌 Sample Metrics Summary:

🔹 Region Dice Scores:
   ✓ ET (Enhancing): {dice_ET:.3f}
   ✓ TC (Tumor Core): {dice_TC:.3f}
   ✓ WT (Whole Tumor): {dice_WT:.3f}
   ✓ Background: {dice_BG:.3f}

🔸 Clinical Acceptance:
   ET > 0.78  → {'⚠️ FAIL' if dice_ET<0.78 else '✅ PASS'}
   TC > 0.70  → {'⚠️ FAIL' if dice_TC<0.70 else '✅ PASS'}
   WT > 0.85  → {'⚠️ FAIL' if dice_WT<0.85 else '✅ PASS'}
"""
plt.text(0.01, 0.5, text, fontsize=10, va='center', bbox=dict(facecolor='#D4EFFC', edgecolor='black'))

plt.tight_layout()
plt.show()


GradCam


In [ ]:
import torch
import numpy as np

# Choose a layer from SwinViT backbone
target_layer = model.swinViT.layers2[0].blocks[-1].norm1

# Containers
gradients = []
activations = []

# Save forward activations
def save_activation(module, inp, out):
    activations.append(out)

# Save backward gradients
def save_gradient(module, grad_in, grad_out):
    gradients.append(grad_out[0])

# Register hooks
target_layer.register_forward_hook(save_activation)
target_layer.register_full_backward_hook(save_gradient)
print("🎯 GradCAM hooks attached to:", target_layer)


In [ ]:
act = activations[-1]
grad = gradients[-1]

# Ensure batch dimension removed
if act.dim() == 5:  # (B, C, D, H, W)
    act = act[0]
    grad = grad[0]

act = act.detach().cpu().numpy()
grad = grad.detach().cpu().numpy()

# Compute channel weights (average gradient per channel)
weights = grad.mean(axis=(1,2,3), keepdims=True)  # collapse D,H,W

# Weighted sum of activations
cam = np.maximum((weights * act).sum(axis=0), 0)

# Normalize heatmap
cam = cam / (cam.max() + 1e-8)

print("🔥 CAM ready, shape:", cam.shape)  # Should be (D,H,W)


In [ ]:
import matplotlib.pyplot as plt

mid = cam.shape[0] // 2  # middle slice

plt.figure(figsize=(6,6))
plt.imshow(cam[mid], cmap='jet')
plt.colorbar()
plt.title("GradCAM Heatmap (Mid Slice)")
plt.show()


In [ ]:
# ===============================================
# 📌 GRADCAM + MULTI-CLASS EXPLAINABILITY (WT-TC-ET)
# ===============================================

import torch, numpy as np, cv2, matplotlib.pyplot as plt

# -----------------------------------------------
# 🔗 Attach Grad-CAM Hooks (do this ONCE)
# -----------------------------------------------
target_layer = model.swinViT.layers2[0].blocks[-1].norm1  # Good mid-depth CAM layer

gradients = []
activations = []

def save_activation(module, inp, out):
    activations.append(out)

def save_gradient(module, grad_in, grad_out):
    gradients.append(grad_out[0])

target_layer.register_forward_hook(save_activation)
target_layer.register_full_backward_hook(save_gradient)

print("🎯 GradCAM attached to:", target_layer)

# -----------------------------------------------------
# 🧠 Function to generate CAM for a given class label
#   Class indices:
#      1 = WT (Whole Tumor)
#      2 = TC (Tumor Core)
#      3 = ET (Enhancing Tumor)
# -----------------------------------------------------
def build_cam(prob_output, cls_index):
    model.zero_grad()
    score = prob_output[cls_index].mean()
    score.backward(retain_graph=True)

    act = activations[-1]
    grad = gradients[-1]

    # Remove batch if present
    if act.dim() == 5:
        act = act[0]
        grad = grad[0]

    act = act.detach().cpu().numpy()
    grad = grad.detach().cpu().numpy()

    weights = grad.mean(axis=(1,2,3), keepdims=True)  # average gradient per channel
    cam = np.maximum((weights * act).sum(axis=0), 0)  # weighted sum
    cam = cam / (cam.max() + 1e-8)  # normalize 0-1
    return cam  # shape: (D, H, W)

# -----------------------------------------------------
# 🎨 Overlay utility: CAM + MRI Slice
# -----------------------------------------------------
def overlay(cam, base_img):
    base = cv2.normalize(base_img, None, 0, 255, cv2.NORM_MINMAX)
    base = np.uint8(base)

    cam_resized = cv2.resize(cam, (base.shape[1], base.shape[0]))

    heat = cv2.applyColorMap(np.uint8(cam_resized * 255), cv2.COLORMAP_JET)
    heat = cv2.cvtColor(heat, cv2.COLOR_BGR2RGB)

    base_rgb = np.stack([base]*3, axis=-1)
    return cv2.addWeighted(base_rgb, 0.6, heat, 0.4, 0)

# -----------------------------------------------------
# 🧪 Generate CAM for a sample from Validation
# -----------------------------------------------------
sample = next(iter(val_loader))
img = sample["image"].to(device)
gt = sample["seg"][0,0].cpu().numpy()  # ground truth segmentation

model.eval()
output = model(img)
prob = output.softmax(1)[0]  # class probabilities

# Compute CAMs for WT, TC, ET
cam_WT = build_cam(prob, 1)
cam_TC = build_cam(prob, 2)
cam_ET = build_cam(prob, 3)

# Pick middle slice
slice_id = cam_WT.shape[0] // 2
mri_slice = img[0,0, slice_id].detach().cpu().numpy()  # FLAIR view for visualization
pred_slice = torch.argmax(prob, 0).cpu().numpy()[slice_id]

# Overlay CAMs on MRI
ov_WT = overlay(cam_WT[slice_id], mri_slice)
ov_TC = overlay(cam_TC[slice_id], mri_slice)
ov_ET = overlay(cam_ET[slice_id], mri_slice)

# -----------------------------------------------------
# 📌 DISPLAY ALL VISUALS
# -----------------------------------------------------
plt.figure(figsize=(14,10))

plt.subplot(2,3,1); plt.title("MRI Slice"); plt.imshow(mri_slice, cmap='gray'); plt.axis('off')
plt.subplot(2,3,2); plt.title("Ground Truth"); plt.imshow(gt[slice_id], cmap='tab20'); plt.axis('off')
plt.subplot(2,3,3); plt.title("Prediction"); plt.imshow(pred_slice, cmap='tab20'); plt.axis('off')

plt.subplot(2,3,4); plt.title("WT CAM Overlay"); plt.imshow(ov_WT); plt.axis('off')
plt.subplot(2,3,5); plt.title("TC CAM Overlay"); plt.imshow(ov_TC); plt.axis('off')
plt.subplot(2,3,6); plt.title("ET CAM Overlay"); plt.imshow(ov_ET); plt.axis('off')

plt.tight_layout()
plt.show()
